<center><h1> Lab04 sheet </h1></center>

**Student ID: 22670287** <br>
**Student Name: Xiaoyan Huang**<br>
**Date created: 28/04/2020**<br>
**Last modified: 4/05/2020**<br>

The main goal of Project2 is to train two Random Forest Regressors using the original data and using the reduced-dimensional one.

# 2. Project2

## 2.1 Preprocess data

In [1]:
import pandas as pd
colnames = ["Sex","Length","Diameter continuous","Height","Whole weight","Shucked weight","Viscera weight","Shell weight","Rings"]
abalone = pd.read_csv("abalone.data", header = None, sep = ',', names = colnames)

In [ ]:
abalone.head()

In [ ]:
abalone.shape

In [ ]:
sex_index = ["M","F","I"]
abalone['Sex'] = abalone['Sex'].apply(sex_index.index)

In [ ]:
abalone.head()

In [ ]:
X = abalone.iloc[:,:8]
y = abalone['Rings']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.15, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train.describe()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
xlab = list(X.columns)
ss = np.arange(len(xlab))
plt.bar(ss, X_train.mean(axis=0))
plt.xticks(ss, xlab, rotation = 45)
plt.ylabel('Mean')
plt.title('the mean of each feature')
plt.show()

- Firstly, the categories *Sex* should be convert into numeric data 0,1 and 2.
- The adalone data is split into training set and test set by a 85/15 proportion. There are 3550 instances in training set and 627 instances in test set.
- From the plot, the mean of feature always between 0 and 1. Thus, there is no feature scaling in the step.


## 2.2 RandomForest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rnd_clf = RandomForestRegressor(n_estimators=200, random_state=42, max_features=0.8, max_samples=3540)

In [ ]:
rnd_clf.fit(X_train, y_train)

In [ ]:
prd_value_test = rnd_clf.predict(X_test)
prd_value_train = rnd_clf.predict(X_train)

In [ ]:
import numpy as np
int_prd_value_test = np.around(prd_value_test)
int_prd_value_train = np.around(prd_value_train)

- There are some hyperparameter values setting. For example, changing the values of *max_features* (the features are considered at each split) and *max_samples* (the number jof samples to draw from X to train each base estimator) can avoid overfitting. It could reduce the effect of outliers.
- Since the predictive values are fraction, using `np.around` converts output into integer for both training and test set.

### 2.2.1 Inspect Mean Square Error

In [ ]:
from sklearn.metrics import mean_squared_error
mse_test = mean_squared_error(y_test,int_prd_value_test)
print("Mean square error of RF model in test data is %s " % mse_test)

In [ ]:
mse_train = mean_squared_error(y_train, int_prd_value_train)
print("Mean square error of RF model in training data is %s " % mse_train)

In [ ]:
import matplotlib.pyplot as plt
mse_list = [mse_test, mse_train]
xlab = ('MSE-test','MSE-train')
ss = np.arange(len(xlab))
plt.bar(ss, mse_list)
plt.xticks(ss, xlab)
plt.ylabel('MSE')
plt.title('MSE of the predictions on training and test set')
plt.show()

In [ ]:
y.value_counts().sort_index()

- The mean square error in test set is pretty higher than training set. The average prediction *Rings* is always higher or lower by 4.7 in test set, but it is higher or lower by 0.7 in training set. However, from the range of *Rings* in the whole dataset is between 1 and 29. Perhaps it is normal to calculate the MSE more than 4. 

### 2.2.2 Inspect Raw Errors

In [ ]:
errors_test = pd.DataFrame()
errors_train = pd.DataFrame()
errors_test['predictive'] = int_prd_value_test
errors_train['predictive'] = int_prd_value_train
errors_test['y_test'] = y_test.tolist()
errors_train['y_test'] = y_train.tolist()
errors_test['errors'] = errors_test['predictive'] - errors_test['y_test']
errors_train['errors'] = errors_train['predictive'] - errors_train['y_test']

In [ ]:
cnt_test = errors_test['errors'].value_counts()
cnt_test = cnt_test.sort_index(ascending=True)
cnt_test

In [ ]:
xlab = cnt_test.sort_index(ascending=True).index
ss = np.arange(len(xlab))
plt.bar(ss, cnt_test)
plt.xticks(ss, xlab, rotation=45)
plt.ylabel('Count')
plt.title('the raw errors on test set')
plt.show()

In [ ]:
cnt_train = errors_train['errors'].value_counts()
cnt_train = cnt_train.sort_index(ascending=True)
cnt_train

In [ ]:
xlab = cnt_train.index
ss = np.arange(len(xlab))
plt.bar(ss, cnt_train)
plt.xticks(ss, xlab)
plt.ylabel('MSE')
plt.title('the prediction errors on train set')
plt.show()

- In terms of test set, the RF regressor only correctly predicts the *Rings* 149 times. There are enormous samples overestimated by one (155 times) and underestimated by one (99 times). Even worse, the RF regressor overestimated and underestimated by over 4.7 (4 times and 24 times).
- For the training set, the RF regressor performs well. It correctly predicts the quality rating with 2086 times, and overestimates (or underestimates) the quality rating by one (or minus one) 763 times (or 471 times).
- However, there is an extremely value -7.

## 2.3 RandomForest Regressor with Reduce Dimension

In [ ]:
feature_list = pd.DataFrame()
feature_list['name'] = colnames[:8]
feature_list['value'] = rnd_clf.feature_importances_

In [ ]:
feature_list = feature_list.sort_values(by = "value", ascending=False)

In [ ]:
s = 0
new_feature = []
for i in range(0, len(feature_list.iloc[:,1])):
    if s < 0.95:
        s = s + feature_list.iloc[i,1]
        new_feature.append(feature_list.iloc[i,0])
new_feature

In [ ]:
X_test_rd = X_test.loc[:,new_feature]
X_train_rd = X_train.loc[:,new_feature]

In [ ]:
X_test_rd.shape, X_train_rd.shape

- Using a loop retains just over 95% of the feature importance. And here, the feature *Sex* is removed due to low feature importance.
- The variable *new_feature* is created, which uses to build a new RF regressor.

In [ ]:
rd_rnd_clf = RandomForestRegressor(n_estimators=150, random_state=42, max_depth=30, max_features=0.8, max_samples=3540)
rd_rnd_clf.fit(X_train_rd, y_train)

In [ ]:
rd_prd_value_test = rd_rnd_clf.predict(X_test_rd)
rd_prd_value_train = rd_rnd_clf.predict(X_train_rd)

In [ ]:
rd_int_prd_test = np.around(rd_prd_value_test)
rd_int_prd_train = np.around(rd_prd_value_train)

- The new RF regressor with reducing the feature dimension is built `rd_rnd_clf`. 
- Some hyperparameters are reset. The value of *n_estimators* (the number of trees in the forest) should be decreased because the performance of previous model is not good in test set. Besides, other hyperparameters retain the same value.

### 2.3.1 Inspect Mean Square Error

In [ ]:
rd_mse_test = mean_squared_error(y_test, rd_int_prd_test)
print("Mean square error of final model in test data is %s " % rd_mse_test)

In [ ]:
rd_mse_train = mean_squared_error(y_train, rd_int_prd_train)
print("Mean square error of final model in training data is %s " % rd_mse_train)

In [ ]:
mse_list = [rd_mse_test, rd_mse_train]
xlab = ('MSE-test','MSE-train')
ss = np.arange(len(xlab))
plt.bar(ss, mse_list)
plt.xticks(ss, xlab)
plt.ylabel('MSE')
plt.title('MSE of the predictions on training and test set')
plt.show()

- The mean square error in test set and training set both increases slightly. And the average prediction Rings is always higher or lower by 4.9 in test set. In terms of MSE, the performances of both regressors are similar.

### 2.3.2 Inspect Raw Errors

In [ ]:
rd_errors_test = pd.DataFrame()
rd_errors_train = pd.DataFrame()
rd_errors_test['predictive'] = rd_int_prd_test
rd_errors_train['predictive'] = rd_int_prd_train
rd_errors_test['y_test'] = y_test.tolist()
rd_errors_train['y_test'] = y_train.tolist()
rd_errors_test['errors'] = rd_errors_test['predictive'] - rd_errors_test['y_test']
rd_errors_train['errors'] = rd_errors_train['predictive'] - rd_errors_train['y_test']

In [ ]:
cnt_test2 = rd_errors_test['errors'].value_counts()
cnt_test2 = cnt_test2.sort_index(ascending=True)
cnt_test2

In [ ]:
xlab = cnt_test2.index
ss = np.arange(len(xlab))
plt.bar(ss, cnt_test2)
plt.xticks(ss, xlab, rotation=45)
plt.ylabel('Count')
plt.title('the raw errors on train set')
plt.show()

In [ ]:
cnt_train2 = rd_errors_train['errors'].value_counts()
cnt_train2 = cnt_train2.sort_index(ascending=True)
cnt_train2

In [ ]:
xlab = cnt_train2.index
ss = np.arange(len(xlab))
plt.bar(ss, cnt_train2)
plt.xticks(ss, xlab)
plt.ylabel('Count')
plt.title('the raw errors on train set')
plt.show()

- In terms of test set, although the RF regressor correctly predicts the Rings 156 times, it is higher than previous one (149 times). The second regressor avoids overfitting to slightly increase the accuracy, even though there are still enormous samples overestimated by one (150 times) and underestimated by one (88 times). Besides, the number of samples overestimated (and underestimated) by over 4.8 (and -4.8) are 4 times (and 26 times).
- For the training set, the RF regressor performs not as good as the previous one. It correctly predicts the quality rating with 2058 times (2086 in the first regressor), and overestimates (or underestimates) the quality rating by one (or minus one) 790 times (or 472 times).
- However, there is an extremely value -12 in test set.

## 2.4 Comparison

In [ ]:
mse_list = [mse_test, mse_train,rd_mse_test, rd_mse_train]
xlab = ('MSE-test','MSE-train','MSE-test-rd','MSE-train-rd')
ss = np.arange(len(xlab))
plt.bar(ss, mse_list)
plt.xticks(ss, xlab)
plt.ylabel('MSE')
plt.title('MSE of the predictions on training and test set')
plt.show()

- For **MSEs**, there are slightly increase after reducing dimension. In terms of **raws error**, the second RF regressor correctly predicts the Rings 156 times higher than the previous one (149 times) and the accuracy increases slightly. But, it also appears extremely error value like -12 in the test set. Thus, there's a tradeoff.